# SCRAPING

## User Profile Scrape using Instaloader

In [ ]:
# pip install instaloader
# !pip install beautifulsoup4

In [ ]:
import instaloader
import time
import os

In [ ]:
loader = instaloader.Instaloader()

In [ ]:
USERNAME = "ahd.iiy"
PASSWORD = "M.Wafiyulahdi123"

In [ ]:
try:
    loader.load_session_from_file(USERNAME)
    print("✅ Sesi login berhasil dimuat.")
except FileNotFoundError:
    print("🔐 Tidak ditemukan sesi login, melakukan login manual...")
    loader.login(USERNAME, PASSWORD)
    loader.save_session_to_file()
    print("✅ Login berhasil dan sesi disimpan.")

In [ ]:
usernames = df_comment['comment_user'].dropna().unique()
user_data = []

In [ ]:
for uname in usernames:
    # Hapus '@' dari username
    username_clean = uname.replace("@", "").strip()
    
    try:
        profile = instaloader.Profile.from_username(loader.context, username_clean)
        user_info = {
            "username": uname,
            "full_name": profile.full_name,
            "followers": profile.followers,
            "followees": profile.followees,
            "is_private": profile.is_private,
            "is_verified": profile.is_verified,
            "bio": profile.biography,
            "post_count": profile.mediacount
        }
        user_data.append(user_info)
        print(f"✅ Data {username_clean} berhasil diambil.")
        time.sleep(3)  # jeda agar tidak rate-limited
    except Exception as e:
        print(f"❌ Gagal mengambil data {username_clean}: {e}")
        user_data.append({
            "username": uname,
            "full_name": None,
            "followers": None,
            "followees": None,
            "is_private": None,
            "is_verified": None,
            "bio": None,
            "post_count": None
        })

# 4. Buat DataFrame hasil scraping user
df_users = pd.DataFrame(user_data)
df_users

## Post and Comment Scrappe using Selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [ ]:
# Ambil semua link postingan
options = Options()
# options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--log-level=3")

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Login ke Instagram
driver.get("https://www.instagram.com/accounts/login/")
time.sleep(5)

username = "username"
password = "pass"

driver.find_element(By.NAME, "username").send_keys(username)
driver.find_element(By.NAME, "password").send_keys(password)
driver.find_element(By.XPATH, "//button[@type='submit']").click()
time.sleep(10)

# Buka profil target
target_username = "yumboxindonesia"
base_url = f"https://www.instagram.com/{target_username}/"
driver.get(base_url)
time.sleep(10)

# Scroll untuk memuat semua post
SCROLL_PAUSE_TIME = 20
NUM_SCROLLS = 10
for _ in range(NUM_SCROLLS):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)

# Ambil semua link post unik
post_links = driver.find_elements(By.XPATH, '//a[contains(@href, "/p/")]')
post_urls = list(set(link.get_attribute('href') for link in post_links if link.get_attribute('href')))

# Simpan link ke file
with open("data_link_post.txt", "w", encoding="utf-8") as f:
    for url in post_urls:
        f.write(url + "\n")

print(f"✅ {len(post_urls)} link berhasil disimpan di data_link_post.txt.")

# Tahap 2: Ambil caption dan komentar dari tiap postingan
with open("data_link_post.txt", "r", encoding="utf-8") as f:
    post_urls = [line.strip() for line in f.readlines() if line.strip()]

with open("komentar.txt", "w", encoding="utf-8") as f:
    for idx, post_url in enumerate(post_urls):
        driver.get(post_url)
        time.sleep(5)
        f.write(f"📌 Post {idx+1} - URL: {post_url}\n")

        # Tambahkan ini sebelum loop komentar dimulai
        wait = WebDriverWait(driver, 10)

        # Di dalam loop saat mengambil komentar:
        try:
            # Tunggu maksimal 10 detik sampai elemen <h1> caption muncul
            h1_element = wait.until(EC.presence_of_element_located((
                By.XPATH,
                '//article//ul//div[1]//li//div//div//div[2]//div[1]//h1'
            )))

            html_caption = h1_element.get_attribute("innerHTML")

            # Parsing HTML caption
            soup = BeautifulSoup(html_caption, "html.parser")

            for br in soup.find_all("br"):
                br.replace_with("\n")

            caption_text = soup.get_text(strip=True)

            f.write(f"📄 Caption: {caption_text}\n")
        except Exception as e:
            f.write("📄 Caption: Tidak ditemukan\n")

        # Klik semua "Lihat semua komentar" jika ada
        while True:
            try:
                load_more = driver.find_element(By.XPATH, "//button[contains(text(),'Lihat semua') or contains(text(),'View all')]")
                driver.execute_script("arguments[0].click();", load_more)
                time.sleep(2)
            except:
                break

        # Tunggu agar komentar dimuat
        
        time.sleep(3)

        # Ambil semua elemen komentar
        comment_items = driver.find_elements(By.XPATH, '//ul[contains(@class, "")]/div/li')

        comment_counter = 1
        for item in comment_items:
            try:
                username_elem = item.find_element(By.XPATH, './/h3//a')
                username = username_elem.get_attribute('href').split('/')[-2]

                comment_elem = item.find_element(By.XPATH, './/span[@dir="auto"]')
                comment_text = comment_elem.text.strip()

                if comment_text:
                    f.write(f"Komentar {comment_counter}: @{username}: {comment_text}\n")
                    comment_counter += 1
            except Exception as e:
                continue
        
         # Tambahan: Likes dan tanggal upload
        try:
            like_elem = driver.find_element(By.XPATH, '//section[2]//div//span/a/span/span')
            like_count = like_elem.text.strip()
            f.write(f"❤️ Likes: {like_count}\n")
        except Exception as e:
            f.write("❤️ Likes: Tidak ditemukan\n")

        try:
            time_elem = driver.find_element(By.XPATH, '//time')
            upload_date = time_elem.get_attribute('datetime')
            f.write(f"📅 Tanggal Upload: {upload_date}\n")
        except Exception as e:
            f.write("📅 Tanggal Upload: Tidak ditemukan\n")

        f.write("---\n\n")

print("✅ Semua komentar berhasil disimpan di komentar.txt")
driver.quit()

# PROCESSING SCRAPE COMMENT

## Selenium

Choose One ----------------------------------------

In [ ]:
with open('komentar.txt', 'r', encoding='utf-8') as f:
    content = f.read()

In [ ]:
# # Buka dua file dan baca isinya
# with open('komentar_skiphop.txt', 'r', encoding='utf-8') as f1, \
#      open('komentar_yumboxindonesia.txt', 'r', encoding='utf-8') as f2:
#     content1 = f1.read()
#     content2 = f2.read()

# # Gabungkan isi kedua file
# content = content1 + '\n' + content2

In [ ]:
# content

f-------------------------------------------------------------

In [ ]:
posts = content.split('---')
data = []

In [ ]:
for post in posts:
    if not post.strip():
        continue

    full_text = post.strip().replace('\n', ' ')

    # URL
    url_match = re.search(r'URL:\s*(https://www\.instagram\.com/\S+)', full_text)
    url = url_match.group(1) if url_match else ''

    # Caption
    caption_match = re.search(r'📄 Caption:\s*(.*?)(?=❤️ Likes:|📅 Tanggal Upload:)', full_text)
    caption = caption_match.group(1).strip() if caption_match else ''

    # Likes
    likes_match = re.search(r'❤️ Likes:\s*(\d+)', full_text)
    likes = likes_match.group(1) if likes_match else '0'

    # Date
    date_match = re.search(r'📅 Tanggal Upload:\s*([\d\-T:.Z]+)', full_text)
    date = date_match.group(1) if date_match else ''

    # Comments
    comment_lines = re.findall(r'Komentar \d+:\s*(.+?)\s*(?=Komentar \d+:|❤️ Likes:|📅 Tanggal Upload:|$)', full_text)
    comment_count = len(comment_lines)

    # Format output
    if comment_lines:
        for i, comment in enumerate(comment_lines):
            user_match = re.match(r'(@[^:]+):', comment)
            comment_user = user_match.group(1) if user_match else ''
            comment_text = comment.split(":", 1)[1].strip() if ":" in comment else comment
            if i == 0:
                data.append([url, caption, date, likes, comment_count, comment_text, comment_user])
            else:
                data.append([''] * 5 + [comment_text, comment_user])
    else:
        data.append([url, caption, date, likes, comment_count, '', ''])

In [ ]:
# Buat DataFrame
df_comment = pd.DataFrame(data, columns=["url", "caption", "dates", "likes count", "comment count", "comments", "comment_user"])
df_comment

## Apify

In [1]:
import pandas as pd

In [2]:
# pip install openpyxl

In [ ]:
df = pd.read_csv('file.csv')

# Mengganti nama kolom
df_comment = df.rename(columns={
    'ownerUsername': 'username',
    'text': 'comments'
})

In [ ]:
df_comment

# DATA ANALYSIS

## Data Cleaning & Transforming

In [96]:
import pandas as pd
import re
import numpy as np
from googletrans import Translator

In [97]:
translator = Translator()

In [98]:
def remove_emojis(text):
    emoji_pattern = re.compile(
        "[" 
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        u"\u2600-\u26FF"
        u"\u2700-\u27BF"
        u"\uFE0F"
        u"\U0001F900-\U0001F9FF"
        u"\U0001FA70-\U0001FAFF"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
emoji_meanings = {
    "❤️": "love it",
    "etc": "etc"
}

In [100]:
def is_emoji_only(text):
    """Cek apakah teks hanya berisi emoji (dan spasi)"""
    text_no_space = text.replace(" ", "")
    return all(char in emoji_meanings for char in text_no_space)

In [ ]:
word_corrections = {
    "jastip": "jasa titip",
    "etc": "etc"
}

In [102]:
def remove_repeated_trailing_chars(word):
    return re.sub(r'([^0-9])\1+', r'\1', word)

In [103]:
# def clean_comment(comment):
#     if not isinstance(comment, str) or comment.strip() == '':
#         return None

#     # Hapus emoji
#     comment = remove_emojis(comment)

#     # Hapus karakter aneh/non-ASCII
#     comment = re.sub(r'[^\x00-\x7F]+', ' ', comment)
    
#     comment = comment.lower()

#     # Hapus huruf double di akhir kata
#     words = comment.split()
#     cleaned_words = [remove_repeated_trailing_chars(word) for word in words]
#     cleaned = ' '.join(cleaned_words)

#     # Koreksi kata tidak baku
#     for wrong, correct in word_corrections.items():
#         cleaned = re.sub(rf'\b{wrong}\b', correct, cleaned)

#     # Lowercase sebelum translate
#     cleaned = cleaned.lower()

#     # Translate
#     try:
#         translated = translator.translate(cleaned.strip(), src='id', dest='en').text
#     except:
#         translated = cleaned.strip()

#     return translated if translated.strip() != '' else None

In [104]:
def clean_comment(comment):
    if not isinstance(comment, str) or comment.strip() == '':
        return None

    comment = comment.strip()

    # 1. Cek jika komentar hanya berisi emoji yang dikenal
    if is_emoji_only(comment):
        words = [emoji_meanings[char] for char in comment if char in emoji_meanings]
        return ' '.join(words) if words else None

    # 2. Proses normal (jika bukan hanya emoji)
    # Hapus emoji
    comment = remove_emojis(comment)

    # Hapus karakter aneh/non-ASCII
    comment = re.sub(r'[^\x00-\x7F]+', ' ', comment)
    
    comment = comment.lower()

    # Hapus huruf double di akhir kata
    words = comment.split()
    cleaned_words = [remove_repeated_trailing_chars(word) for word in words]
    cleaned = ' '.join(cleaned_words)

    # Koreksi kata tidak baku
    for wrong, correct in word_corrections.items():
        cleaned = re.sub(rf'\b{wrong}\b', correct, cleaned)

    # Translate
    try:
        translated = translator.translate(cleaned.strip(), src='id', dest='en').text
    except:
        translated = cleaned.strip()

    return translated if translated.strip() != '' else None

In [ ]:
# Step 1: Ambil kolom comments
sa_comments = df_comment['comments'].copy()

# Step 2: Hapus kosong
sa_comments = sa_comments.replace('', np.nan).dropna().reset_index(drop=True)

# Step 3: Konversi ke string
sa_comments = sa_comments.astype(str)

# Step 4: Jalankan fungsi cleaning
sa_comments = sa_comments.apply(clean_comment)

# Step 5: Hapus hasil kosong setelah cleaning
sa_comments = sa_comments.replace('', np.nan).dropna().reset_index(drop=True)

# Step 6: Lakukan lowercase SETELAH translate
sa_comments = sa_comments.apply(lambda x: x.lower())

# Step 6.5: Hapus komentar yang hanya terdiri dari 1-2 huruf
sa_comments = sa_comments[sa_comments.apply(lambda x: len(x.strip()) > 2)].reset_index(drop=True)

# Step 7 (Opsional): Simpan ke CSV
sa_comments.to_csv("sa_comments.csv", sep=";", index=False, encoding='utf-8')

# Preview hasil
sa_comments.head()

## Sentiment Analysis

In [106]:
# pip install nltk
# !pip install wordcloud

In [ ]:
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from nltk.sentiment import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

In [108]:
def get_sentiment_label(text):
    score = sia.polarity_scores(text)['compound']
    if score >= 0.05:
        return 'positive'
    elif score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

In [109]:
# Buat DataFrame baru
sa_df = pd.DataFrame({'comment': sa_comments})

# Tambahkan kolom 'sentiment'
sa_df['sentiment'] = sa_df['comment'].apply(get_sentiment_label)

# Preview hasil
# print(sa_df.head())
# sa_df
sa_df.to_csv("sa_mooij_df.csv", index=False, encoding='utf-8-sig')

In [ ]:
# Hitung jumlah komentar per label sentimen
sentiment_counts = sa_df['sentiment'].value_counts().reindex(['positive', 'neutral', 'negative'])

# Plot
plt.figure(figsize=(8, 5))
sns.barplot(x=sentiment_counts.index, y=sentiment_counts.values, palette='Set2')

plt.title("Jumlah Komentar Berdasarkan Sentimen", fontsize=14)
plt.xlabel("Label Sentimen")
plt.ylabel("Jumlah Komentar")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
from collections import Counter

# Gabungkan semua komentar menjadi satu string
all_text = " ".join(sa_df['comment'])

# Pisahkan kata dan hitung frekuensinya
words = all_text.split()
word_freq = Counter(words)

# Ubah ke DataFrame dan urutkan
word_freq_df = pd.DataFrame(word_freq.items(), columns=['Word', 'Frequency'])
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)
word_freq_df.to_csv("word_freq_df_mooij.csv", index=False, encoding='utf-8-sig')

# Buat WordCloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_text)

# Tampilkan WordCloud
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Word Cloud Semua Komentar", fontsize=16)
plt.show()